In [1]:
! pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 16.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.27.1
    Uninstalling huggingface-hub-0.27.1:
      Successfully uninstalled huggingface-hub-0.27.1


## Transformers

Though we have GPU power, but as we have quite short text fields, we will rather use sentence transformer, also for making experiments quicker. 

In [16]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cuda')
model.half()
embeddings = model.encode(sentences)
print(len(embeddings[0]))


384


In [6]:
import polars as pl

df = pl.read_csv("../data/filtered/movies.csv")

In [11]:
# combine tagline and description into one text field
df = df.with_columns(
    pl.concat_str(pl.col('tagline'), pl.col('description'), separator=' ').alias('text')
)

In [35]:
text_list = df['text'].to_list()
embeddings = model.encode(text_list)
df = df.with_columns(
    pl.Series("text_embeddings", embeddings.tolist()[0])
)

AttributeError: 'LinearRegression' object has no attribute 'encode'

In [36]:
df.row(1)

(1,
 1000004,
 'Fight Club',
 1999,
 'Mischief. Mayhem. Soap.',
 'A ticking-time-bomb insomniac and a slippery soap salesman channel primal male aggression into a shocking new form of therapy. Their concept catches on, with underground "fight clubs" forming in every town, until an eccentric gets in the way and ignites an out-of-control spiral toward oblivion.',
 139,
 'R',
 4.27,
 'Mischief. Mayhem. Soap. A ticking-time-bomb insomniac and a slippery soap salesman channel primal male aggression into a shocking new form of therapy. Their concept catches on, with underground "fight clubs" forming in every town, until an eccentric gets in the way and ignites an out-of-control spiral toward oblivion.',
 [-0.06121826171875,
  -0.00634765625,
  -0.004726409912109375,
  0.008453369140625,
  0.01248931884765625,
  -0.0006728172302246094,
  0.1322021484375,
  0.0238189697265625,
  -0.027557373046875,
  -0.0201873779296875,
  0.0129547119140625,
  -0.0163726806640625,
  0.0278778076171875,
  0.04

In [54]:
df_train = df.filter(pl.col('date') < 2015)
df_val = df.filter(pl.col('date') >= 2015)

X_train = df_train.select('text_embeddings').to_numpy()
y_train = df_train.select('rating').to_numpy()

X_val = df_val.select('text_embeddings').to_numpy()
y_val = df_val.select('rating').to_numpy()


In [55]:
X_train = [x[0] for x in X_train]
X_test = [x[0] for x in X_val]

X_train = np.array(X_train)
X_test = np.array(X_test)


X_train.shape

(9094, 384)

In [49]:
X_test[0]


np.float64(4.27)

In [58]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.metrics import mean_squared_error


models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=0.1),
}

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    print(f"{model_name} RMSE: {rmse}")

Linear Regression RMSE: 0.5764623933833773
Ridge Regression RMSE: 0.5763471346635302


# Conclusion:
text only model performs worse than numerical model (lacks more context about movie).